<a href="https://colab.research.google.com/github/20520566-uit/IE212_LAB-20520566/blob/main/IE212_LAB03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("DS200_Lab3") \
    .getOrCreate()

1. Hãy đọc dữ liệu từ các file csv, sử dụng tự suy ra kiểu dữ liệu cho mỗi cột.

In [16]:
path = "/content/data/"
orders = spark.read.csv(path + "Orders.csv", header=True, inferSchema=True, sep=';')
customers = spark.read.csv(path + "Customer_List.csv", header=True, inferSchema=True, sep=';')
order_items = spark.read.csv(path + "Order_Items.csv", header=True, inferSchema=True, sep=';')
products = spark.read.csv(path + "Products.csv", header=True, inferSchema=True, sep=';')
reviews = spark.read.csv(path + "Order_Reviews.csv", header=True, inferSchema=True, sep=';')

orders.printSchema()

root
 |-- Order_ID: string (nullable = true)
 |-- Customer_Trx_ID: string (nullable = true)
 |-- Order_Status: string (nullable = true)
 |-- Order_Purchase_Timestamp: timestamp (nullable = true)
 |-- Order_Approved_At: timestamp (nullable = true)
 |-- Order_Delivered_Carrier_Date: timestamp (nullable = true)
 |-- Order_Delivered_Customer_Date: timestamp (nullable = true)
 |-- Order_Estimated_Delivery_Date: timestamp (nullable = true)



2. Thống kê tổng số đơn hàng, số lượng khách hàng và người bán.

In [17]:
total_orders = orders.select("Order_ID").distinct().count()
total_customers = customers.select("Subscriber_ID").distinct().count()
total_sellers = order_items.select("Seller_ID").distinct().count()

print("Tổng số đơn hàng:", total_orders)
print("Tổng số khách hàng:", total_customers)
print("Tổng số người bán:", total_sellers)

Tổng số đơn hàng: 99441
Tổng số khách hàng: 99382
Tổng số người bán: 3095


3. Phân tích số lượng đơn hàng theo quốc gia, sắp xếp theo thứ tự giảm dần.

In [19]:
orders_country = orders.join(customers, "Customer_Trx_ID")

orders_country.groupBy("Customer_Country") \
    .count() \
    .orderBy(desc("count")) \
    .show()

+----------------+-----+
|Customer_Country|count|
+----------------+-----+
|         Germany|41754|
|          France|12848|
|     Netherlands|11629|
|         Belgium| 5464|
|         Austria| 5043|
|     Switzerland| 3640|
|  United Kingdom| 3382|
|          Poland| 2139|
|         Czechia| 2034|
|           Italy| 2025|
|           Spain| 1651|
|        Portugal| 1336|
|          Sweden|  975|
|         Denmark|  905|
|          Serbia|  746|
|          Norway|  716|
|        Slovakia|  534|
|        Slovenia|  495|
|          Turkey|  485|
|          Greece|  412|
+----------------+-----+
only showing top 20 rows


4. Phân tích số lượng đơn hàng nhóm theo năm, tháng đặt hàng (Hiển thị theo năm
tăng dần, tháng giảm dần)

In [20]:
orders_time = orders.withColumn(
    "year", year("order_purchase_timestamp")
).withColumn(
    "month", month("order_purchase_timestamp")
)

orders_time.groupBy("year", "month") \
    .count() \
    .orderBy(asc("year"), desc("month")) \
    .show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2022|   12|    1|
|2022|   10|  324|
|2022|    9|    4|
|2023|   12| 5673|
|2023|   11| 7544|
|2023|   10| 4631|
|2023|    9| 4285|
|2023|    8| 4331|
|2023|    7| 4026|
|2023|    6| 3245|
|2023|    5| 3700|
|2023|    4| 2404|
|2023|    3| 2682|
|2023|    2| 1780|
|2023|    1|  800|
|2024|   10|    4|
|2024|    9|   16|
|2024|    8| 6512|
|2024|    7| 6292|
|2024|    6| 6167|
+----+-----+-----+
only showing top 20 rows


5. Thống kê điểm đánh giá trung bình, số lượng đánh giá theo từng mức (ví dụ: 1 đến
5).

In [22]:
clean_reviews = reviews \
    .filter(col("review_score").isNotNull()) \
    .filter(col("review_score").between(1, 5))

In [26]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, avg, when, regexp_extract, rlike

# Regular expression to match integers (positive or negative, optional leading/trailing spaces)
numeric_regex = "^\\s*-?\\d+\\s*$"

# Safely cast Review_Score to IntegerType, setting non-numeric values to NULL
reviews_with_numeric_score = reviews.withColumn(
    "review_score_numeric",
    when(col("Review_Score").rlike(numeric_regex), col("Review_Score").cast(IntegerType()))
    .otherwise(None)
)

# Filter out null scores and scores outside the 1-5 range
clean_reviews_for_avg = reviews_with_numeric_score \
    .filter(col("review_score_numeric").isNotNull()) \
    .filter(col("review_score_numeric").between(1, 5))

# Calculate and show the average review score
print("Average Review Score:")
clean_reviews_for_avg.select(avg("review_score_numeric")).show()

# Calculate and show review counts by score level
print("\nReview counts by score level:")
clean_reviews_for_avg.groupBy("review_score_numeric") \
    .count() \
    .orderBy("review_score_numeric") \
    .show()

Average Review Score:
+-------------------------+
|avg(review_score_numeric)|
+-------------------------+
|       4.0864214950162765|
+-------------------------+


Review counts by score level:
+--------------------+-----+
|review_score_numeric|count|
+--------------------+-----+
|                   1|11424|
|                   2| 3151|
|                   3| 8179|
|                   4|19141|
|                   5|57328|
+--------------------+-----+



6. Tính doanh thu (giá sản phẩm + phí vận chuyển) trong năm 2024 và nhóm theo
danh mục sản phẩm

In [27]:
orders_2024 = orders.filter(year("order_purchase_timestamp") == 2024)

revenue_2024 = orders_2024 \
    .join(order_items, "order_id") \
    .join(products, "product_id") \
    .withColumn("revenue", col("price") + col("freight_value"))

revenue_2024.groupBy("product_category_name") \
    .agg(round(sum("revenue"), 2).alias("total_revenue")) \
    .orderBy(desc("total_revenue")) \
    .show()

+---------------------+-------------+
|product_category_name|total_revenue|
+---------------------+-------------+
|        Health_Beauty|    885191.12|
|        Watches_Gifts|    771986.75|
|       Bed_Bath_Table|     650794.7|
|       Sports_Leisure|    621999.34|
| Computers_Accesso...|    594771.04|
|           Housewares|    491576.96|
|      Furniture_Decor|    476466.13|
|                 Auto|    404210.57|
|                 Baby|    299052.56|
|           Cool_Stuff|    273910.05|
|         Garden_Tools|    259068.32|
|            Telephony|    217452.13|
|            Perfumery|    204562.54|
|                 Toys|    200634.07|
|     Office_Furniture|    181745.73|
|           Stationery|    164743.85|
|             Pet_Shop|    152804.94|
| Construction_Tool...|    141187.34|
|          Electronics|    134265.45|
|  Musical_Instruments|    121476.31|
+---------------------+-------------+
only showing top 20 rows


7. Xác định sản phẩm có số lượng bán ra cao nhất và tính điểm đánh giá trung bình
cho từng sản phẩm

In [28]:
product_sales = order_items.groupBy("product_id") \
    .agg(count("*").alias("total_sold"))

product_reviews = reviews.join(order_items, "order_id") \
    .groupBy("product_id") \
    .agg(avg("review_score").alias("avg_rating"))

result = product_sales.join(product_reviews, "product_id") \
    .orderBy(desc("total_sold"))

result.show()

+--------------------+----------+------------------+
|          product_id|total_sold|        avg_rating|
+--------------------+----------+------------------+
|aca2eb7d00ea1a7b8...|       527| 4.019083969465649|
|99a4788cb24856965...|       488|3.8983402489626555|
|422879e10f4668299...|       484|3.9465020576131686|
|389d119b48cf3043d...|       392| 4.117647058823529|
|368c6c730842d7801...|       388| 3.922680412371134|
|53759a2ecddad2bb8...|       373| 3.868632707774799|
|d1c427060a0f73f6b...|       343| 4.194117647058824|
|53b36df67ebb7c415...|       323|          4.190625|
|154e7e31ebfa09220...|       281| 4.315068493150685|
|3dd2a17168ec895c7...|       274| 4.209558823529412|
|2b4609f8948be1887...|       260| 4.070631970260223|
|7c1bd920dbdf22470...|       231| 3.876595744680851|
|a62e25e09e05e6faf...|       226|3.8577777777777778|
|5a848e4ab52fd5445...|       197|4.1179487179487175|
|bb50f2e236e5eea01...|       195| 4.224489795918367|
|e0d64dcfaa3b6db5c...|       194|  3.777202072

8. Tính toán hiệu số giữa ngày giao hàng thực tế (Order_Delivered_Carrier_Date) và
ngày giao hàng dự kiến (ví dụ: Shipping_Limit_Date từ bảng Order_Items) để
đánh giá hiệu suất giao hàng.

In [29]:
delivery_perf = orders.join(order_items, "order_id") \
    .withColumn(
        "delay_days",
        datediff(col("order_delivered_carrier_date"), col("shipping_limit_date"))
    )

delivery_perf.select(
    avg("delay_days").alias("avg_delay_days"),
    max("delay_days").alias("max_delay_days")
).show()

+-------------------+--------------+
|     avg_delay_days|max_delay_days|
+-------------------+--------------+
|-3.4358580964685617|           118|
+-------------------+--------------+



9. Nhóm khách hàng dựa trên số lượng đơn hàng, giá trị trung bình của đơn hàng và
tần suất mua sắm.

In [35]:
customer_behavior = orders.join(order_items, "order_id") \
    .groupBy("Customer_Trx_ID") \
    .agg(
        count("order_id").alias("order_count"),
        avg("price").alias("avg_order_value")
    )

customer_behavior.show()

+--------------------+-----------+---------------+
|     Customer_Trx_ID|order_count|avg_order_value|
+--------------------+-----------+---------------+
|f54a9f0e6b351c431...|          1|           19.9|
|2a1dfb647f32f4390...|          1|          445.0|
|4f28355e5c17a4a42...|          1|          139.0|
|4632eb5a8f175f6fe...|          1|           79.9|
|843ff05b30ce4f75b...|          1|           84.9|
|a4156bb8aff5d6722...|          1|          174.9|
|1099d033c74a027a7...|          1|           49.9|
|7a3bd3b37285f0ab2...|          4|           99.0|
|5fff39f1b59dc4d2f...|          2|          24.49|
|18f6ca10777417c93...|          2|          14.89|
|f1e46939e6408b3e6...|          1|           37.9|
|a8695124db570d100...|          1|          69.99|
|545b9a267af9ba134...|          1|         116.99|
|90d7075599361b694...|          2|          27.03|
|a340ce6c3570e68d4...|          1|          99.49|
|ce0681e1c3f70e145...|          1|           71.0|
|ee905ec97794ec6e9...|         

10. Xếp hạng các seller dựa trên tổng doanh thu và số lượng đơn hàng bán được.

In [34]:
seller_rank = order_items \
    .withColumn("revenue", col("price") + col("freight_value")) \
    .groupBy("seller_id") \
    .agg(
        sum("revenue").alias("total_revenue"),
        count("order_id").alias("total_orders")
    ) \
    .orderBy(desc("total_revenue"))

seller_rank.show()

+--------------------+------------------+------------+
|           seller_id|     total_revenue|total_orders|
+--------------------+------------------+------------+
|4869f7a5dfa277a7d...|249640.69999999992|        1156|
|7c67e1448b00f6e96...|239536.44000000006|        1364|
|53243585a1d6dc264...| 235856.6800000001|         410|
|4a3ca9315b744ce9f...| 235539.9599999999|        1987|
|fa1c13f2614d7b5c4...| 204084.7300000001|         586|
|da8622b14eb17ae28...|185192.32000000018|        1551|
|7e93a43ef30c4f03f...|182754.05000000005|         340|
|1025f0e2d44d7041d...|         172860.69|        1428|
|7a67c85e85bb2ce85...| 162648.3799999999|        1171|
|955fee9216a65b617...|         160602.68|        1499|
|6560211a19b47992c...|151265.76999999987|        2033|
|1f50f920176fa81da...|142104.98000000033|        1931|
|46dc3b2cc0980fb8e...|139909.69000000003|         542|
|a1043bafd471dff53...|133745.24999999994|         770|
|cc419e0650a3c5ba7...| 129957.4099999996|        1775|
|620c87c17